In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
from monty.serialization import loadfn, dumpfn

In [3]:
from ase.filters import FrechetCellFilter
from ase.optimize import LBFGS
from pymatgen.io.ase import AseAtomsAdaptor
# from orb_models.forcefield import pretrained
# from orb_models.forcefield.calculator import ORBCalculator
from fairchem.core import pretrained_mlip, FAIRChemCalculator
# from eqnorm.calculator import EqnormCalculator
# from mace.calculators import mace_mp
# from mattersim.forcefield import MatterSimCalculator
# from sevenn.calculator import SevenNetCalculator


/home/atshaam/miniconda3/envs/proj_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
MODEL = "uma" # "orb" or "uma" or "mace" or "seven" or "mattersim" or "eqnorm"

In [5]:
supercell_size = 10

In [6]:
# orb_ff = pretrained.orb_v3_conservative_inf_omat(device="cuda",precision="float32-high")  # or "float32-highest" / "float64
# orb_calc = ORBCalculator(orb_ff, device="cuda")

uma_ff = pretrained_mlip.get_predict_unit("uma-m-1p1", device="cuda")
uma_calc = FAIRChemCalculator(uma_ff, task_name="omat")

# eqnorm_calc = EqnormCalculator(model_name='eqnorm', model_variant='eqnorm-mptrj', device='cuda')


# mace_calc = mace_mp(model="MACE-matpes-r2scan-omat-ft.model", dispersion=False, device='cuda')
# seven_calc = SevenNetCalculator('7net-mf-ompa', modal='omat24')

# mattersim_calc = MatterSimCalculator(device="cuda")

MODELS = {
    # "orb":orb_calc,
    "uma": uma_calc,
    # # "seven":seven_calc,
    # "mattersim":mattersim_calc,
    # "eqnorm":eqnorm_calc
    # "mace": mace_calc
}

In [7]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [8]:
def relax_structure(structure): # takes in pymatgen structure
    atoms = structure.to_ase_atoms()
    atoms.calc = MODELS[MODEL]
    
    opt = LBFGS(FrechetCellFilter(atoms))# Delete this FrechetCellFilter when doing the defect structures, need FrechetCellFilter when doing competing phases
    opt.run(0.01, 1000)
    
    structure1 = AseAtomsAdaptor.get_structure(atoms)
    energy = atoms.get_potential_energy()
    energy_per_atom = energy / len(atoms)

    return structure1, energy_per_atom

In [9]:
from doped.chemical_potentials import CompetingPhases
from pymatgen.entries.computed_entries import ComputedEntry
from doped.chemical_potentials import CompetingPhasesAnalyzer
from tqdm import tqdm
from pymatgen.entries.compatibility import MaterialsProject2020Compatibility

In [10]:
from pymatgen.core import Element

MP_U_VALUES = {
    "Co": 3.32,
    "Cr": 3.7,
    "Fe": 5.3,
    "Mn": 3.98,
    "Mo": 4.38,
    "Ni": 6.2,
    "V": 3.25,
    "W": 6.2,
}

from pymatgen.core import Species

def add_parameters(entry):
    parameters = {}
    if any([Element(el) in entry.composition.elements for el in MP_U_VALUES]):
        parameters["run_type"] = "GGA+U"
        parameters["hubbards"] = {el.name: MP_U_VALUES.get(el.symbol, 0.0) for el in entry.composition.elements}
        parameters["is_hubbard"] = True
    else:
        parameters["run_type"] = "GGA"
        parameters["hubbards"] = None
        parameters["is_hubbard"] = False
    entry.parameters = parameters
    return entry

In [11]:
calculated_entries = {}

def calc_competing_phase(comp): # Pass in formula of species
    cp = CompetingPhases(composition=comp, extrinsic='Cr', energy_above_hull=0.05)
    
    entries = []
    for entry in tqdm(cp.entries):
        key =  entry.data['material_id'] + entry.structure.composition.reduced_formula
        
        if key in calculated_entries:
            new_entry = calculated_entries[key]
        else:
            try:
                relaxed_structure, energy = relax_structure(entry.structure)
            except ValueError as e:
                print(f"Skipping structure due to error: {e}")
                continue
            new_entry = ComputedEntry(
                relaxed_structure.composition,
                energy, 
                data = {'material_id':entry.data['material_id']}
            )
            new_entry.structure = relaxed_structure   # IMPORTANT!
            new_entry = add_parameters(new_entry)     # IMPORTANT!
            calculated_entries[key] = new_entry 
        entries.append(new_entry)
    compat = MaterialsProject2020Compatibility(check_potcar=False, check_potcar_hash=False)
    new_entries = compat.process_entries(entries, inplace=False, on_error="warn")
    return cp, new_entries 

In [12]:
def calc_competing_phase_analysis(comp, entries):
    
    cpa = CompetingPhasesAnalyzer(composition=comp, entries=entries)
    
    return cpa.chempots, cpa.unstable_host

In [13]:
relaxed_defect = loadfn(f'../../data/{MODEL}_corrected_doped_defect_energies_per_atom_{supercell_size}A.json')

In [17]:
new = ['mp-19521', 'mp-3081']

In [14]:
for key in relaxed_defect.keys():
    if key != 'mp-755882':
        formula = relaxed_defect[key]['formula']
        cp, entries = calc_competing_phase(formula) # do you have to return cp?
        chem_pot, unstable = calc_competing_phase_analysis(formula, entries)
        if 'competing phases' not in relaxed_defect[key]:
            relaxed_defect[key]['competing phases'] = {}
        relaxed_defect[key]['competing phases']['chemical potentials'] = chem_pot
        relaxed_defect[key]['competing phases']['entries'] = entries
        relaxed_defect[key]['competing phases']['unstable'] = unstable

Duplicate entry names found for generated competing phases: AlPO4_Pbca_EaH_0.012!
Duplicate entry names found for generated competing phases: AlPO4_Pbca_EaH_0.012!
  0%|                                                                                            | 0/62 [00:00<?, ?it/s]

       Step     Time          Energy          fmax
LBFGS:    0 21:30:36     -381.049002        0.037276


logm result may be inaccurate, approximate err = 4.726475621142667e-13


LBFGS:    1 21:30:36     -381.049357        0.035889


logm result may be inaccurate, approximate err = 4.677408557278127e-13


...

 43%|████████████████████████████████████                                               | 10/23 [00:20<00:28,  2.19s/it]

       Step     Time          Energy          fmax
LBFGS:    0 02:04:18      -14.200245        0.044525
LBFGS:    1 02:04:19      -14.200322        0.036516
LBFGS:    2 02:04:19      -14.200481        0.000157


 48%|███████████████████████████████████████▋                                           | 11/23 [00:21<00:24,  2.03s/it]

       Step     Time          Energy          fmax
LBFGS:    0 02:04:20      -56.752723        0.052451
LBFGS:    1 02:04:20      -56.752834        0.049119
LBFGS:    2 02:04:20      -56.753676        0.035837
LBFGS:    3 02:04:21      -56.753769        0.030152
LBFGS:    4 02:04:21      -56.753877        0.006074


100%|███████████████████████████████████████████████████████████████████████████████████| 23/23 [00:23<00:00,  1.02s/it]
Failed to guess oxidation states for Entry None (Al45Cr7). Assigning anion correction to only the most electronegative atom.
Invalid U value of 3.7 on Cr, expected 0.0 for {'@module': 'pymatgen.entries.computed_entries', '@class': 'ComputedEntry', 'energy': -4.639466689035062, 'composition': {'Al': 45.0, 'Cr': 7.0}, 'entry_id': None, 'correction': 0.0, 'energy_adjustments': [], 'parameters': {'run_type': 'GGA+U', 'hubbards': {'Al': 0.0, 'Cr': 3.7}, 'is_hubbard': True}, 'data': {'material_id': 'mp-31019', 'oxidation_states': {}}}
BeAl2O4 is not stable with respect to competing phases, having an energy above hull of 2.4468 eV/atom.
Formally, this means that (based on the supplied athermal calculation data) the host material is unstable and so has no chemical potential limits; though in reality the host may be stabilised by temperature effects etc, or just a metastable 

In [19]:
for key in relaxed_defect.keys():
    print(relaxed_defect[key].keys())

dict_keys(['supercell', 'Cr_Al_0', 'formula', 'minimum energy site', 'competing phases'])
dict_keys(['supercell', 'Cr_Al_0', 'formula', 'minimum energy site', 'competing phases'])
dict_keys(['supercell', 'Cr_Al_0', 'formula', 'minimum energy site', 'competing phases'])
dict_keys(['supercell', 'Cr_Al_0', 'formula', 'minimum energy site', 'competing phases'])
dict_keys(['supercell', 'Cr_Al_0', 'formula', 'minimum energy site', 'competing phases'])
dict_keys(['supercell', 'Cr_Al_C1_O1.87O1.90O1.94_0', 'Cr_Al_C1_O1.87O1.90O1.95_0', 'formula', 'minimum energy site', 'competing phases'])
dict_keys(['supercell', 'Cr_Al_0', 'formula', 'minimum energy site', 'competing phases'])
dict_keys(['supercell', 'Cr_Al_0', 'formula', 'minimum energy site', 'competing phases'])
dict_keys(['supercell', 'Cr_Al_0', 'formula', 'minimum energy site', 'competing phases'])
dict_keys(['supercell', 'Cr_Al_0', 'formula', 'minimum energy site', 'competing phases'])
dict_keys(['supercell', 'Cr_Al_0', 'formula', 'mini

In [15]:
dumpfn(relaxed_defect, f'../../data/{MODEL}_corrected_doped_defect_energies_per_atom_{supercell_size}A.json')

## Leave

In [20]:
ex_cp = loadfn(f'../data/{MODEL}_doped_defect_energies_{supercell_size}A.json')

In [21]:
supercell_10A = loadfn(f'../data/{MODEL}_doped_defect_energies_10A.json')

In [24]:
for key, val in ex_cp.items():
    ex_cp[key]['minimum energy site']['energy'] = supercell_10A[key]['minimum energy site']['energy']
    ex_cp[key]['minimum energy site']['information'] = supercell_10A[key]['minimum energy site']['information']
    ex_cp[key]['supercell']['energy'] = supercell_10A[key]['supercell']['energy']

In [25]:
dumpfn(ex_cp, f'../data/{MODEL}_doped_defect_energies_10A.json')